In [2]:
import pandas as pd
import json


In [3]:
with open('final_label_json_data.json') as f:
    sentiment = pd.DataFrame(json.load(f))
    
sentiment.head()    

,amazon_score_mixed,amazon_score_negative,amazon_score_neutral,amazon_score_positive,amazon_sentiments_label,azure_api_label,azure_api_score,google_sentiment_label,google_sentiment_magnitude,google_sentiment_socre,ibm_score,ibm_sentiments_label,sentiment,text
0,0.001549,0.000533,0.750581,0.247337,neutral,positive,0.978328,positive,1.1,0.2,0.816136,positive,positive,"Thank you. Good afternoon, everyone. And welco..."
1,0.009229,0.006072,0.880566,0.104133,neutral,neutral,0.500000,positive,1.4,0.2,0.558518,positive,positive,"As we look past Q1, we expect the channel inve..."
2,0.000831,0.008268,0.980631,0.010270,neutral,neutral,0.500000,neutral,0.0,0.0,0.000000,neutral,neutral,And your last question comes from the line of ...
3,0.011638,0.953385,0.034877,0.000099,negative,neutral,0.500000,negative,1.2,-0.3,-0.598559,negative,negative,"On the China gaming weakness, is it the slower..."
4,0.014957,0.000221,0.007328,0.977495,positive,positive,0.905933,positive,7.9,0.5,0.790615,positive,negative,"I don't know that we could tear that apart, te..."


In [4]:
sentiment['amazon_sentiment_score'] = sentiment[["amazon_score_negative","amazon_score_neutral","amazon_score_positive"]].max(axis=1)

In [5]:
sentiment.head()

,amazon_score_mixed,amazon_score_negative,amazon_score_neutral,amazon_score_positive,amazon_sentiments_label,azure_api_label,azure_api_score,google_sentiment_label,google_sentiment_magnitude,google_sentiment_socre,ibm_score,ibm_sentiments_label,sentiment,text,amazon_sentiment_score
0,0.001549,0.000533,0.750581,0.247337,neutral,positive,0.978328,positive,1.1,0.2,0.816136,positive,positive,"Thank you. Good afternoon, everyone. And welco...",0.750581
1,0.009229,0.006072,0.880566,0.104133,neutral,neutral,0.500000,positive,1.4,0.2,0.558518,positive,positive,"As we look past Q1, we expect the channel inve...",0.880566
2,0.000831,0.008268,0.980631,0.010270,neutral,neutral,0.500000,neutral,0.0,0.0,0.000000,neutral,neutral,And your last question comes from the line of ...,0.980631
3,0.011638,0.953385,0.034877,0.000099,negative,neutral,0.500000,negative,1.2,-0.3,-0.598559,negative,negative,"On the China gaming weakness, is it the slower...",0.953385
4,0.014957,0.000221,0.007328,0.977495,positive,positive,0.905933,positive,7.9,0.5,0.790615,positive,negative,"I don't know that we could tear that apart, te...",0.977495


In [6]:
#Keeping only the columns we need for mapping the label we assigned it with the scores of all 4 API's. 
sentiment_df = sentiment[['azure_api_score', 'google_sentiment_socre','ibm_score','amazon_sentiment_score','sentiment']]
sentiment_df.head()

,azure_api_score,google_sentiment_socre,ibm_score,amazon_sentiment_score,sentiment
0,0.978328,0.2,0.816136,0.750581,positive
1,0.500000,0.2,0.558518,0.880566,positive
2,0.500000,0.0,0.000000,0.980631,neutral
3,0.500000,-0.3,-0.598559,0.953385,negative
4,0.905933,0.5,0.790615,0.977495,negative


In [7]:
sentiment_df.shape

(1644, 5)

In [8]:
#Removing the neutral sentiments for our model

sentiment_df = sentiment_df[sentiment_df['sentiment'] != 'neutral']
sentiment_df.head()

,azure_api_score,google_sentiment_socre,ibm_score,amazon_sentiment_score,sentiment
0,0.978328,0.2,0.816136,0.750581,positive
1,0.500000,0.2,0.558518,0.880566,positive
3,0.500000,-0.3,-0.598559,0.953385,negative
4,0.905933,0.5,0.790615,0.977495,negative
5,0.904133,0.0,0.988573,0.674756,positive


In [9]:
sentiment_df.shape

(811, 5)

In [10]:
#Build a model to map the output Sentiement label with the sentiment scores from all 4 apis

#label encode the output variable
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
sentiment_df['sentiment'] = le.fit_transform(sentiment_df['sentiment'])

In [11]:
sentiment_df.head()

,azure_api_score,google_sentiment_socre,ibm_score,amazon_sentiment_score,sentiment
0,0.978328,0.2,0.816136,0.750581,1
1,0.500000,0.2,0.558518,0.880566,1
3,0.500000,-0.3,-0.598559,0.953385,0
4,0.905933,0.5,0.790615,0.977495,0
5,0.904133,0.0,0.988573,0.674756,1


In [12]:
from sklearn.preprocessing import MinMaxScaler


In [13]:

X = sentiment_df.iloc[:,:-1]
Y = sentiment_df.iloc[:,-1]
features = X.columns.values

features

array(['azure_api_score', 'google_sentiment_socre', 'ibm_score',
       'amazon_sentiment_score'], dtype=object)

In [14]:
X.head()

,azure_api_score,google_sentiment_socre,ibm_score,amazon_sentiment_score
0,0.978328,0.2,0.816136,0.750581
1,0.500000,0.2,0.558518,0.880566
3,0.500000,-0.3,-0.598559,0.953385
4,0.905933,0.5,0.790615,0.977495
5,0.904133,0.0,0.988573,0.674756


In [15]:
Y.head()

0    1
1    1
3    0
4    0
5    1
Name: sentiment, dtype: int64

In [16]:
#Scaling all variables to a range of 0 to 1
sc = MinMaxScaler(feature_range= (0,1))
X = pd.DataFrame(sc.fit_transform(X), columns= features)
#features = X.columns 

In [17]:
X.head()

,azure_api_score,google_sentiment_socre,ibm_score,amazon_sentiment_score
0,0.980185,0.5625,0.903736,0.646323
1,0.468419,0.5625,0.767878,0.831746
2,0.468419,0.2500,0.157679,0.935622
3,0.902730,0.7500,0.890277,0.970015
4,0.900803,0.4375,0.994673,0.538159


In [18]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

**AutoSKlearn**

In [19]:
!pip install auto-sklearn

  Using cached https://files.pythonhosted.org/packages/3f/e7/7f24ef402a5766c677683e313c5595137d754cb9eb1c99627803280e79d5/numpy-1.14.5-cp37-cp37m-manylinux1_x86_64.whl
  Found existing installation: numpy 1.16.2
    Uninstalling numpy-1.16.2:
      Successfully uninstalled numpy-1.16.2


In [21]:
import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import cross_val_score, train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# automl = autosklearn.classification.AutoSklearnClassifier()
# automl.fit(X_train, Y_train)
# y_hat = automl.predict(X_test)
# print("Accuracy score", sklearn.metrics.accuracy_score(Y_test, y_hat))

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, Y_train)
y_hat = automl.predict(X_test)
print("Accuracy score", sklearn.metrics.accuracy_score(Y_test, y_hat))

/home/jai/miniconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2019-03-21 08:38:02,092:EnsembleBuilder(1):9af0d6fbec1b15d1b06c1bc99a3f82d0] No models better than random - using Dummy Score!
[WARNING] [2019-03-21 08:38:02,100:EnsembleBuilder(1):9af0d6fbec1b15d1b06c1bc99a3f82d0] No models better than random - using Dummy Score!


/home/jai/miniconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/jai/miniconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/jai/miniconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/jai/miniconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/jai/miniconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/jai/miniconda3/lib/python3.7/site-packages/autosklearn/eval

KeyboardInterrupt: 

https://github.com/automl/auto-sklearn/issues/520